In [88]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
from ipywidgets import FloatProgress
from time import sleep
from IPython.display import display
import re
import pickle
from urlextract import URLExtract
import json
import sqlite3
import datetime
from tqdm import tqdm

In [89]:
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}

## Womens Juttis

In [90]:
womens_jutti=[]
for i in range(1,5):
    try:
        result=requests.get(f"https://pastelsandpop.com/collections/juttis?page={i}",headers=headers)
        get=result.content
        data=BeautifulSoup(get,'lxml')
        prod=data.find_all('div',{'class':'grid-product__content'})
        for j in prod:
            womens_jutti.append("https://pastelsandpop.com"+j.find('a').attrs['href'])
    except:
        pass 

In [91]:
len(womens_jutti)

78

In [113]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Cat_list=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [114]:
err=[]
for i in tqdm(womens_jutti):
    try:
        result=requests.get(i,headers=headers)
        get=result.content
        soup=BeautifulSoup(get,'lxml')
        js=json.loads(soup.find('script',{'type':'application/ld+json'}).contents[0])
        product_name=js['name']
        product_brand=js['brand']
        product_link=i
        product_cat='jutti'
        product_des=js['description']
        s=soup.find('span',{'class':'product__price product__price--compare'})
        if s:
            product_mrp=s.find('span',{'class':'money'}).text
            product_mrp=int(''.join(product_mrp[4:].split(',')))
        else:
            product_mrp=int(float(js['offers']['price']))
        product_price=int(float(js['offers']['price']))    
        p=soup.find('div',{'class':'product-section'}).attrs['id'].split('-')[1]
        images=soup.find_all('div',{'class':f"product__thumb product__thumb-{p} js-no-transition"})
        img=[]
        for k in images:
            img.append(k.find('img').attrs['data-src'].split('{width}')[0]+'1024x1024.jpg')
        primary_link=img[0]
        secondary_link=','.join(img[1:])
        sizes=soup.find_all('div',{'class':'variant-input'})
        size=[]
        for i in sizes:
            size.append(i.attrs['data-value'])
        size_avl=','.join(size)
    
    except:
        err.append(i)
        pass
        
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(product_des)
        Gender.append('Women')
        Cat_list.append(product_cat)
        MRP.append(product_mrp)
        Price.append(product_price)
        Primary_Image_Link.append(primary_link)
        Secondary_Image_Link.append(secondary_link)
        Size_Available.append(size_avl)
    

100%|██████████| 78/78 [01:58<00:00,  1.51s/it]


In [115]:
women_data=pd.DataFrame({
    
    "Website":'https://pastelsandpop.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Product_Category":Cat_list,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [116]:
women_data.head()

,Website,Product_Link,Product_Name,Product_Brand,Product_Category,Size_Avail,Price,MRP,Gender,Description,Primary_Image_Links,Secondary_Image_Links,Affiliate_Link
0,https://pastelsandpop.com/,https://pastelsandpop.com/collections/juttis/p...,LAYLA : Pastels and Pop X Anushree Reddy,Pastels and Pop,jutti,"35,36,37,38,39,40,41,42",3690,3690,Women,"Intricate thread embroidery, fashioned from a ...",//cdn.shopify.com/s/files/1/0997/6106/products...,//cdn.shopify.com/s/files/1/0997/6106/products...,
1,https://pastelsandpop.com/,https://pastelsandpop.com/collections/juttis/p...,MEHER : Pastels and Pop X Anushree Reddy,pastels & pop,jutti,"35,36,37,38,39,40,41,42",3590,3590,Women,With a shimmering floral pattern embroidered o...,//cdn.shopify.com/s/files/1/0997/6106/products...,//cdn.shopify.com/s/files/1/0997/6106/products...,
2,https://pastelsandpop.com/,https://pastelsandpop.com/collections/juttis/p...,CELEBRATION : Pastels and Pop X Anushree Reddy,pastels & pop,jutti,"35,36,37,38,39,40,41,42",4290,4290,Women,The showstopper of the Pastels and Pop X Anush...,//cdn.shopify.com/s/files/1/0997/6106/products...,//cdn.shopify.com/s/files/1/0997/6106/products...,
3,https://pastelsandpop.com/,https://pastelsandpop.com/collections/juttis/p...,Rani,Pastels and Pop,jutti,"35,36,37,38,39,40,41,42",3290,3290,Women,If being the belle of every ball is your thing...,//cdn.shopify.com/s/files/1/0997/6106/products...,//cdn.shopify.com/s/files/1/0997/6106/products...,
4,https://pastelsandpop.com/,https://pastelsandpop.com/collections/juttis/p...,GOLD DUST,Pastels and Pop,jutti,"35,36,37,38,39,40,41,42",2800,2800,Women,Looking for that perfect tone of gold that's n...,//cdn.shopify.com/s/files/1/0997/6106/products...,//cdn.shopify.com/s/files/1/0997/6106/products...,


In [117]:
conn = sqlite3.connect(r'p&p.db')
c = conn.cursor()

c.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
conn.commit()

women_data.to_sql('product_details', conn, if_exists='replace', index = False)
conn.close()

## Mens Jutti

In [118]:
mens_jutti=[]
for i in range(1,2):
    try:
        result=requests.get(f"https://pastelsandpop.com/collections/mens-juttis?page={i}",headers=headers)
        get=result.content
        data=BeautifulSoup(get,'lxml')
        prod=data.find_all('div',{'class':'grid-product__content'})
        for j in prod:
            mens_jutti.append("https://pastelsandpop.com"+j.find('a').attrs['href'])
    except:
        pass 

In [119]:
len(mens_jutti)

12

In [120]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Cat_list=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [121]:
err=[]
for i in tqdm(mens_jutti):
    try:
        result=requests.get(i,headers=headers)
        get=result.content
        soup=BeautifulSoup(get,'lxml')
        js=json.loads(soup.find('script',{'type':'application/ld+json'}).contents[0])
        product_name=js['name']
        product_brand=js['brand']
        product_link=i
        product_cat='jutti'
        product_des=js['description']
        s=soup.find('span',{'class':'product__price product__price--compare'})
        if s:
            product_mrp=s.find('span',{'class':'money'}).text
            product_mrp=int(''.join(product_mrp[4:].split(',')))
        else:
            product_mrp=int(float(js['offers']['price']))
        product_price=int(float(js['offers']['price']))
        p=soup.find('div',{'class':'product-section'}).attrs['id'].split('-')[1]
        images=soup.find_all('div',{'class':f"product__thumb product__thumb-{p} js-no-transition"})
        img=[]
        for k in images:
            img.append(k.find('img').attrs['data-src'].split('{width}')[0]+'1024x1024.jpg')
        primary_link=img[0]
        secondary_link=','.join(img[1:])
        sizes=soup.find_all('div',{'class':'variant-input'})
        size=[]
        for i in sizes:
            size.append(i.attrs['data-value'])
        size_avl=','.join(size)
    
    except:
        err.append(i)
        pass
        
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(product_des)
        Gender.append('Men')
        Cat_list.append(product_cat)
        MRP.append(product_mrp)
        Price.append(product_price)
        Primary_Image_Link.append(primary_link)
        Secondary_Image_Link.append(secondary_link)
        Size_Available.append(size_avl)
    

100%|██████████| 12/12 [00:15<00:00,  1.26s/it]


In [122]:
men_data=pd.DataFrame({
    
    "Website":'https://pastelsandpop.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Product_Category":Cat_list,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [123]:
men_data.head()

,Website,Product_Link,Product_Name,Product_Brand,Product_Category,Size_Avail,Price,MRP,Gender,Description,Primary_Image_Links,Secondary_Image_Links,Affiliate_Link
0,https://pastelsandpop.com/,https://pastelsandpop.com/collections/mens-jut...,MAHIWAL,Pastels and Pop,jutti,"5,6,7,8,9,10",3100,3100,Men,Mahiwal is here to give you the assurance to t...,//cdn.shopify.com/s/files/1/0997/6106/products...,//cdn.shopify.com/s/files/1/0997/6106/products...,
1,https://pastelsandpop.com/,https://pastelsandpop.com/collections/mens-jut...,SIKANDAR,Pastels and Pop,jutti,"5,6,7,8,9,10",2800,2800,Men,They say you cannot go wrong with a solid; no ...,//cdn.shopify.com/s/files/1/0997/6106/products...,//cdn.shopify.com/s/files/1/0997/6106/products...,
2,https://pastelsandpop.com/,https://pastelsandpop.com/collections/mens-jut...,RAANJHA,Pastels and Pop,jutti,"5,6,7,8,9,10",2900,2900,Men,One look and you’ll know the Ranjha is all you...,//cdn.shopify.com/s/files/1/0997/6106/products...,//cdn.shopify.com/s/files/1/0997/6106/products...,
3,https://pastelsandpop.com/,https://pastelsandpop.com/collections/mens-jut...,ZAYN,Pastels and Pop,jutti,"5,6,7,8,9,10",3400,3400,Men,Introducing Zayn - our first ever silver Men’s...,//cdn.shopify.com/s/files/1/0997/6106/products...,//cdn.shopify.com/s/files/1/0997/6106/products...,
4,https://pastelsandpop.com/,https://pastelsandpop.com/collections/mens-jut...,TREE OF LIFE,Pastels and Pop,jutti,"5,6,7,8,9,10",3400,3400,Men,Say hello to this handsome new addition to our...,//cdn.shopify.com/s/files/1/0997/6106/products...,//cdn.shopify.com/s/files/1/0997/6106/products...,


In [124]:
conn = sqlite3.connect(r'p&p.db')
c = conn.cursor()

#c.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
#conn.commit()
men_data.to_sql('product_details', conn, if_exists='append', index = False)
conn.close()

## Clothings

In [125]:
clothings=[]
for i in range(1,2):
    try:
        result=requests.get(f"https://pastelsandpop.com/collections/clothing?page={i}",headers=headers)
        get=result.content
        data=BeautifulSoup(get,'lxml')
        prod=data.find_all('div',{'class':'grid-product__content'})
        for j in prod:
            clothings.append("https://pastelsandpop.com"+j.find('a').attrs['href'])
    except:
        pass 

In [126]:
clothings

['https://pastelsandpop.com/collections/clothing/products/beaded-peplum-tank',
 'https://pastelsandpop.com/collections/clothing/products/floral-black-dress',
 'https://pastelsandpop.com/collections/clothing/products/floral-ruffle-sleeve-top',
 'https://pastelsandpop.com/collections/clothing/products/white-lace-top',
 'https://pastelsandpop.com/collections/clothing/products/jumpsuit',
 'https://pastelsandpop.com/collections/clothing/products/gingham-ruffle-tank',
 'https://pastelsandpop.com/collections/clothing/products/ruffle-bow-back-dress']

In [127]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Cat_list=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [128]:
err=[]
for i in tqdm(clothings):
    try:
        result=requests.get(i,headers=headers)
        get=result.content
        soup=BeautifulSoup(get,'lxml')
        js=json.loads(soup.find('script',{'type':'application/ld+json'}).contents[0])
        product_name=js['name']
        product_brand=js['brand']
        product_link=i
        product_cat='clothing'
        product_des=js['description']
        s=soup.find('span',{'class':'product__price product__price--compare'})
        if s:
            product_mrp=s.find('span',{'class':'money'}).text
            product_mrp=int(''.join(product_mrp[4:].split(',')))
        else:
            product_mrp=int(float(js['offers']['price']))
        product_price=int(float(js['offers']['price']))
        p=soup.find('div',{'class':'product-section'}).attrs['id'].split('-')[1]
        images=soup.find_all('div',{'class':f"product__thumb product__thumb-{p} js-no-transition"})
        img=[]
        for k in images:
            img.append(k.find('img').attrs['data-src'].split('{width}')[0]+'1024x1024.jpg')
        primary_link=img[0]
        secondary_link=','.join(img[1:])
        sizes=soup.find_all('div',{'class':'variant-input'})
        size=[]
        for i in sizes:
            size.append(i.attrs['data-value'])
        size_avl=','.join(size)
    
    except:
        err.append(i)
        pass
        
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(product_des)
        Gender.append('Women')
        Cat_list.append(product_cat)
        MRP.append(product_mrp)
        Price.append(product_price)
        Primary_Image_Link.append(primary_link)
        Secondary_Image_Link.append(secondary_link)
        Size_Available.append(size_avl) 
    

100%|██████████| 7/7 [00:12<00:00,  1.75s/it]


In [129]:
clothing=pd.DataFrame({
    
    "Website":'https://pastelsandpop.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Product_Category":Cat_list,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [130]:
clothing.head()

,Website,Product_Link,Product_Name,Product_Brand,Product_Category,Size_Avail,Price,MRP,Gender,Description,Primary_Image_Links,Secondary_Image_Links,Affiliate_Link
0,https://pastelsandpop.com/,https://pastelsandpop.com/collections/clothing...,Beaded Peplum Tank,Pastels and Pop,clothing,"XS,S,M,L",1600,2100,Women,An all-around favorite here at Pastels and Pop...,//cdn.shopify.com/s/files/1/0997/6106/products...,//cdn.shopify.com/s/files/1/0997/6106/products...,
1,https://pastelsandpop.com/,https://pastelsandpop.com/collections/clothing...,Floral Black Dress,Pastels and Pop,clothing,"S,M,XS",1299,1950,Women,A wear-all-season mini dress in a gorgeous flo...,//cdn.shopify.com/s/files/1/0997/6106/products...,//cdn.shopify.com/s/files/1/0997/6106/products...,
2,https://pastelsandpop.com/,https://pastelsandpop.com/collections/clothing...,Floral Ruffle Sleeve Top,Pastels and Pop,clothing,"M,S,XS,L",1150,1500,Women,A zero-effort way to update your jeans-and-a-t...,//cdn.shopify.com/s/files/1/0997/6106/products...,//cdn.shopify.com/s/files/1/0997/6106/products...,
3,https://pastelsandpop.com/,https://pastelsandpop.com/collections/clothing...,White Lace Top,Pastels and Pop,clothing,"XS,S,M",950,1100,Women,A wear-always white top with eyelet lace at th...,//cdn.shopify.com/s/files/1/0997/6106/products...,//cdn.shopify.com/s/files/1/0997/6106/products...,
4,https://pastelsandpop.com/,https://pastelsandpop.com/collections/clothing...,Jaipur Jumpsuit,Pastels and Pop,clothing,"XS,S,M,L",1750,2300,Women,An easy wide-leg jumpsuit with an undeniably p...,//cdn.shopify.com/s/files/1/0997/6106/products...,//cdn.shopify.com/s/files/1/0997/6106/products...,


In [131]:
conn = sqlite3.connect(r'p&p.db')
c = conn.cursor()

#c.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
#conn.commit()

clothing.to_sql('product_details', conn, if_exists='append', index = False)
conn.close()